This notebook is a tutorial for generation of SE descriptor in data2.

#  Load Dependencies

In [4]:
import numpy as np
import pandas as pd
import glob
from rdkit import Chem
from script.utils import Calc_SPMS_Elec,Scaler 
import warnings
warnings.filterwarnings("ignore")

Using backend: pytorch[16:05:40] /opt/dgl/src/runtime/tensordispatch.cc:43: TensorDispatcher: dlopen failed: /root/anaconda3/lib/python3.8/site-packages/dgl/tensoradapter/pytorch/libtensoradapter_pytorch_1.11.0.so: cannot open shared object file: No such file or directory



# Load Data

In [5]:
data_file = './data2/data2.csv'
mol_dir = './data2/data2_sdf_files/'

tmp_df = pd.read_csv(data_file)
cat_smiles = np.array(tmp_df['Catalyst'].to_list())
imine_smiles = np.array(tmp_df['Imine'].to_list())
thiol_smiles = np.array(tmp_df['Thiol'].to_list())
labels = np.array(tmp_df['Output'].to_list())
label_std = (np.array(labels)-np.min(labels))/(np.max(labels)-np.min(labels))
cat_smi_set = list(set(cat_smiles))
imine_smi_set = list(set(imine_smiles))
thiol_smi_set = list(set(thiol_smiles))

In [6]:
with open(mol_dir+'smiles_file_dict.csv','r') as fr:
    lines = fr.readlines()

smiles_name_dict = {tmp_line.strip().split(',')[0]:tmp_line.strip().split(',')[1] for tmp_line in lines}

In [7]:
sort_func = lambda x: eval(x.split('/')[-1].split('.')[0].split('_')[-1])
cat_files = sorted(glob.glob('./data2/data2_sdf_files/cat_*.sdf'),key=sort_func)
imine_files = sorted(glob.glob('./data2/data2_sdf_files/imine_*.sdf'),key=sort_func)
thiol_files = sorted(glob.glob('./data2/data2_sdf_files/thiol_*.sdf'),key=sort_func)

cat_spms,cat_elec_desc,cat_id_name_dict = Calc_SPMS_Elec(cat_files,mol_dir,align='Origin',sphere_radius = 7).calc_spms_elec(cat_files,mol_dir)
imine_spms,imine_elec_desc,imine_id_name_dict = Calc_SPMS_Elec(imine_files,mol_dir,align='Origin',sphere_radius = 7).calc_spms_elec(imine_files,mol_dir)
thiol_spms,thiol_elec_desc,thiol_id_name_dict = Calc_SPMS_Elec(thiol_files,mol_dir,align='Origin',sphere_radius = 7).calc_spms_elec(thiol_files,mol_dir)

cat_spms = Scaler(cat_spms)
imine_spms = Scaler(imine_spms)
thiol_spms = Scaler(thiol_spms)

cat_elec_desc = Scaler(cat_elec_desc)
imine_elec_desc = Scaler(imine_elec_desc)
thiol_elec_desc = Scaler(thiol_elec_desc)

# Generate local steric and electronic information

In [8]:
total_cat_spms = np.array([cat_spms[cat_id_name_dict[smiles_name_dict[tmp_smi]]] for tmp_smi in cat_smiles])
total_imine_spms = np.array([imine_spms[imine_id_name_dict[smiles_name_dict[tmp_smi]]] for tmp_smi in imine_smiles])
total_thiol_spms = np.array([thiol_spms[thiol_id_name_dict[smiles_name_dict[tmp_smi]]] for tmp_smi in thiol_smiles])

total_cat_elec_desc = np.array([cat_elec_desc[cat_id_name_dict[smiles_name_dict[tmp_smi]]] for tmp_smi in cat_smiles])
total_imine_elec_desc = np.array([imine_elec_desc[imine_id_name_dict[smiles_name_dict[tmp_smi]]] for tmp_smi in imine_smiles])
total_thiol_elec_desc = np.array([thiol_elec_desc[thiol_id_name_dict[smiles_name_dict[tmp_smi]]] for tmp_smi in thiol_smiles])

cat_spms_trans = np.transpose(total_cat_spms,(0,2,3,1))
imine_spms_trans = np.transpose(total_imine_spms,(0,2,3,1))
thiol_spms_trans = np.transpose(total_thiol_spms,(0,2,3,1))

cat_elec_trans = np.transpose(total_cat_elec_desc,(0,2,3,4,1))
imine_elec_trans = np.transpose(total_imine_elec_desc,(0,2,3,4,1))
thiol_elec_trans = np.transpose(total_thiol_elec_desc,(0,2,3,4,1))

react_spms = np.concatenate([cat_spms_trans,imine_spms_trans,thiol_spms_trans],axis=3)
react_elec = np.concatenate([cat_elec_trans,imine_elec_trans,thiol_elec_trans],axis=4)
label_std = (np.array(labels)-np.min(labels))/(np.max(labels)-np.min(labels))

In [9]:
np.save('./data2/run_data/data2_SEMG_MIGNN_react_spms.npy',react_spms)
np.save('./data2/run_data/data2_SEMG_MIGNN_react_elec.npy',react_elec)
np.save('./data2/run_data/data2_SEMG_MIGNN_labels.npy',labels)
np.save('./data2/run_data/data2_SEMG_MIGNN_label_std.npy',label_std)